In [80]:
import os 
import json
import pickle
import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd
from pprint import pprint
from bisect import bisect_left as find_prev
import matplotlib.pyplot as plt

import xml.etree.ElementTree as xml

strptime = datetime.datetime.strptime
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# READING 


In [5]:
#df_kept.to_csv('chosen_segments_'+str(cut)+'.csv')


cut=0.5
df_kept=pd.read_csv('chosen_segments_'+str(cut)+'.csv')
df_kept['unique_id']=df_kept['shot_id'].map(str)+'_'+df_kept['file'].map(str)
df_kept['unique_id']=df_kept['unique_id'].str.replace('.mp4','')

In [149]:
df_kept.character.unique()

array(['Peggy Mitchell', 'Tanya Branning', 'Jack Branning',
       'Archie Mitchell', 'Max Branning'], dtype=object)

In [69]:
soap_opera_scale = {'extramarital affair': 1/1.98, 'get divorced': 1/1.96,'illegitimate child': 1/1.45,'institutionalized for emotional problem': 1/1.43,'happily married': 1/4.05,'serious accident': 1/2.96,'murdered': 1/1.81,'attempt suicide': 1/1.26,'blackmailed': 1/1.86,'unfaithful spouse': 1/2.23,'sexually assaulted': 1/2.60,'abortion': 1/1.41}
life_events_scale = {'death relative': 100, 'Divorce':	73, 'Marital separation':	65, 'Imprisonment':	63, 'injury illness':	53, 'Marriage':	50, 'Fired':	47, 'Marital reconciliation':	45, 'Retirement':	45, 'Pregnancy':	40, 'Sexual difficulties':	39, 'New family member':	39, 'Business readjustment':	39, 'Money change':	38, 'Work change': 36, 'Arguing':	35, 'Mortgage':	32, 'Child leaving home':	29, 'Trouble with in-laws':	29,  'Achievement':	28}

In [198]:
# which one is what?
# original soap opera scale
soap_1_path   ='../other_experiments/Hugging_Face_classification/soap_opera_scale_Max_Jack_Tanya.csv' 
soap_2_path   ='../other_experiments/Hugging_Face_classification/soap_opera_Peggy_Archie.csv' 
# on keyword "event"
events_1_path ='../other_experiments/Hugging_Face_classification/events_ranking_Max_Jack_Tanya.csv'
# original life scale ranking (concatenate the first and second)
life_1_path   ='../other_experiments/Hugging_Face_classification/life_scale_ranking_Max_Jack_Tanya.csv'
life_end_path ='../other_experiments/Hugging_Face_classification/life_scale_ranking_end_Max_Jack_Tanya.csv'
# on keyword "daily life"
classification_results_path='../other_experiments/Hugging_Face_classification/daily_life_Max_Jack_Tanya.csv'
# life scale with improved labels (top 20)
reduced_life_clean ='../other_experiments/Hugging_Face_classification/reduced_life_scale_Max_Jack_Tanya _clean.csv'

extra_columns = ['index', 'Unnamed: 0.1', 'Unnamed: 0']


hf_results=pd.read_csv(reduced_life_clean)
hf_results=pd.read_csv(soap_2_path)
hf_results=pd.read_csv(soap_1_path)


hf_results=hf_results.drop(columns=[c for c in extra_columns if c in hf_results.columns])
hf_results['filename']=hf_results['filename'].str.replace('.xml','')
hf_results['unique_id']=hf_results['shot_id'].map(str)+'_'+hf_results['filename'].map(str)
hf_results.head()

,begin,end,filename,shot_id,transcript,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score,unique_id
0,01:23:25:18,01:23:29:11,5253646087419594905,1472,"You must be, giving Stacey up like this.",0.450417,0.489040,0.400918,0.164084,0.000035,0.236502,0.335613,0.076780,0.301274,0.444586,0.272091,0.409387,3.580726,1472_5253646087419594905
1,01:29:59:21,01:30:01:11,5264050645694159353,1658,"Let her go, for God's sake!",0.395017,0.385765,0.433672,0.342158,0.000380,0.261511,0.066604,0.107600,0.201647,0.409807,0.255371,0.393366,3.252897,1658_5264050645694159353
2,00:21:32:15,00:21:37:13,5274434588125702436,365,"Can you tell Bianca, when she peels herself off of her deathbed, that next time, I want a doctor's note.",0.304901,0.001987,0.027043,0.074392,0.000008,0.297072,0.153931,0.785221,0.501472,0.425037,0.161045,0.406486,3.138593,365_5274434588125702436
3,01:17:38:21,01:17:41:19,5245830105934359183,1475,Pat'll kill me! Three weeks. Month.,0.307820,0.246752,0.068564,0.378333,0.000356,0.308431,0.503700,0.241117,0.461864,0.388335,0.089380,0.108456,3.103108,1475_5245830105934359183
4,00:58:50:06,00:58:54:19,5258818087037440299,1171,"No, you've gotta promise me you won't say a word. Right, my lips are sealed.",0.288786,0.306285,0.247446,0.283195,0.011572,0.270417,0.159182,0.327130,0.528973,0.281832,0.148655,0.228945,3.082417,1171_5258818087037440299


In [199]:
hf_results.filename.unique()

array(['5253646087419594905', '5264050645694159353',
       '5274434588125702436', '5245830105934359183',
       '5258818087037440299', '5248439298566680341',
       '5256228221757951173', '5269245838135402929',
       '5271844722846213628', '5266645664934403562',
       '5251033029316736209'], dtype=object)

In [200]:
hf_results.describe()

,shot_id,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,1046.603744,0.079765,0.036035,0.037144,0.056932,0.010463,0.050365,0.013417,0.027656,0.054147,0.101968,0.019161,0.017775,0.504828
std,611.979444,0.087468,0.060510,0.062810,0.067725,0.028429,0.067323,0.043930,0.047907,0.093380,0.105755,0.036322,0.035600,0.439787
min,2.000000,0.000009,0.000014,0.000027,0.000039,0.000003,0.000011,0.000013,0.000015,0.000024,0.000015,0.000009,0.000018,0.000463
25%,525.250000,0.018018,0.003596,0.005885,0.013248,0.000153,0.003279,0.000574,0.002677,0.007343,0.021682,0.002183,0.001682,0.169379
50%,1030.000000,0.051304,0.015048,0.016407,0.035368,0.001148,0.020438,0.002250,0.011935,0.018605,0.065580,0.007551,0.007270,0.386581
75%,1553.750000,0.109284,0.041432,0.041000,0.075847,0.006443,0.071675,0.008151,0.033341,0.050277,0.145983,0.019920,0.020000,0.715170
max,2531.000000,0.503533,0.508568,0.686225,0.688697,0.245816,0.337179,0.548935,0.785221,0.535093,0.446623,0.382038,0.705860,3.580726


In [201]:
if 'extramarital affair' in hf_results.columns:
    for c in hf_results.columns:
        if c in soap_opera_scale:
            hf_results[c] = hf_results[c] / soap_opera_scale[c]
elif 'death relative' in hf_results.columns:
    for c in hf_results.columns:
        if c in life_events_scale:
            hf_results[c] = hf_results[c] / life_events_scale[c]

In [202]:
hf_results.describe()

,shot_id,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,1046.603744,0.157935,0.070628,0.053858,0.081413,0.042376,0.149080,0.024285,0.034847,0.100714,0.227389,0.049819,0.025062,0.504828
std,611.979444,0.173187,0.118600,0.091074,0.096847,0.115139,0.199276,0.079513,0.060363,0.173686,0.235834,0.094438,0.050196,0.439787
min,2.000000,0.000018,0.000028,0.000039,0.000056,0.000013,0.000031,0.000024,0.000019,0.000045,0.000033,0.000023,0.000025,0.000463
25%,525.250000,0.035676,0.007048,0.008534,0.018944,0.000618,0.009707,0.001040,0.003373,0.013659,0.048351,0.005675,0.002372,0.169379
50%,1030.000000,0.101583,0.029494,0.023791,0.050576,0.004649,0.060495,0.004073,0.015039,0.034605,0.146243,0.019633,0.010251,0.386581
75%,1553.750000,0.216383,0.081206,0.059449,0.108462,0.026095,0.212158,0.014753,0.042010,0.093516,0.325543,0.051793,0.028200,0.715170
max,2531.000000,0.996995,0.996793,0.995026,0.984837,0.995553,0.998049,0.993573,0.989379,0.995273,0.995970,0.993299,0.995263,3.580726


In [203]:
# hf_results.sort_values(by=['extramarital affair'],ascending=False).head(3)

In [204]:
apply_coefficients = True
transform = np.log

cols_to_drop=[ 'shot_id', 'filename', 'transcript','score',
              'Marital separation','New family member','Business readjustment', 
              'Money change', 'Work change', 'Arguing','Mortgage', 'Child leaving home', 
              'Trouble with in-laws', 'Achievement', 'happily married']#

min_max_scaler  = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

chosen_cols = [c for c in hf_results.columns if c in ['death relative', 'Divorce', 'Imprisonment', 'injury illness', 'Marriage','Fired', 'Marital reconciliation', 'Retirement', 'Pregnancy','Sexual difficulties', 
                                                      'extramarital affair', 'get divorced', 'illegitimate child', 'institutionalized for emotional problem', 'serious accident', 'murdered', 'attempt suicide', 'blackmailed', 'unfaithful spouse', 'sexually assaulte', 'abortion']]

final_results = hf_results.copy()
final_results[chosen_cols] = robust_scaler.fit_transform(final_results[chosen_cols])

if apply_coefficients:
    for c in hf_results.columns:
        if 'extramarital affair' in hf_results.columns:
            if c in soap_opera_scale:
                hf_results[c] = hf_results[c] * transform(soap_opera_scale[c])
        elif 'death relative' in hf_results.columns:
            if c in life_events_scale:
                hf_results[c] = hf_results[c] * transform(life_events_scale[c])

In [205]:
final_results['score_max'] = final_results.drop([c for c in cols_to_drop if c in final_results.columns], axis=1).max(axis=1)

final_results = final_results.sort_values(by=['score_max'],ascending=False)

filtered_results = pd.merge(final_results,df_kept,on='unique_id')

filtered_result_char=filtered_results[filtered_results.character == 'Archie Mitchell']
filtered_result_char=filtered_result_char[filtered_result_char.transcript.str.split().apply(len)>5]

filtered_result_char[['transcript', 'unique_id'] + chosen_cols + ['score_max']].head(20)

,transcript,unique_id,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion,score_max
10,"It's Jase's funeral today, innit. Oh. I'm sorry, mate.",525_5256228221757951173,-0.559417,-0.357280,-0.418663,0.717864,2.536726,57.133414,-0.360513,-0.376568,0.509470,-0.382270,57.133414
24,"Someone dragged her out on a shopping spree, brought her ankles up like a pair of barrage balloons.",341_5258818087037440299,1.845684,-0.367249,-0.278349,-0.516078,4.339358,28.995979,-0.015353,3.937974,0.888145,-0.390813,28.995979
113,"It's a myth, innit? It never works. Hair of the dog? No, just numbs the pain. Puts it all off. So, when do you reckon you're gonna get the all clear then, Pat?",1258_5251033029316736209,0.201744,0.085299,0.707858,1.155411,0.576379,0.509920,11.629984,0.569446,0.255425,2.230594,11.629984
129,"And who can blame her, hm? You lost, Sean.",612_5271844722846213628,2.437888,10.968048,3.882135,0.291937,2.222344,10.007503,3.071549,1.103614,2.756261,4.136762,10.968048
130,Just as long as I toe the Mitchell line?,1977_5266645664934403562,-0.529117,-0.245268,-0.413054,0.287808,-0.248146,-0.229557,-0.329855,10.960768,-0.374114,0.076514,10.960768
175,"Now you want her out clubbing. She wanted to come. Yeah, well, pregnant girls don't always know what's best for them, do they? Anyway, she said you needed some moral support, so, er, here I am. You'll have to wait. We're busy.",342_5258818087037440299,0.215429,-0.393802,-0.207911,-0.231909,-0.259988,-0.144042,-0.372476,9.187420,-0.192490,-0.310435,9.187420
176,"You're gonna sue her? Bang go our savings. Marinas are expensive. How about up the river? Ah, now that's a good idea. Let's moor it next to HMS Belfast(!)",1439_5245830105934359183,2.224924,1.895092,-0.157599,0.645353,1.914728,1.524454,-0.313540,9.138681,2.593961,-0.270520,9.138681
190,"Actually Jean, they won't be joining us.",1783_5266645664934403562,0.837585,8.670743,3.830933,2.824035,1.900543,4.915113,4.428493,1.457389,1.326158,5.019159,8.670743
222,"No, Archie, I don't wanna go. No, no, no, look. Just for him. Show support. You know, 12 months inside is no joke.",1807_5245830105934359183,0.090669,0.044648,3.946641,4.908729,3.397239,7.793180,7.008032,0.969404,0.916512,0.487150,7.793180
242,"Where d'you think you're going? To find my baby. No listen. You need some rest. Yeah, Ronnie's right, darlin'. No. I can't wait any more.",442_5271844722846213628,-0.022702,-0.284735,7.318971,0.436091,1.277035,0.658150,-0.288904,0.280989,0.513911,0.586691,7.318971


In [206]:
n_words = 10
for character in ['Peggy Mitchell', 'Archie Mitchell', 'Max Branning', 'Jack Branning', 'Tanya Branning']:
    filtered_result_char=filtered_results[filtered_results.character == character]
    # filtered_result_char=filtered_result_char[filtered_result_char.transcript.str.split().apply(len)>n_words]

    print(len(filtered_result_char), 'lines for', character)
    
    top20 = filtered_result_char[['transcript', 'filename', 'unique_id', 'begin', 'end',] + chosen_cols + ['score_max']]
    top20.to_csv('final_'+character+'_nominwords.csv')

277 lines for Peggy Mitchell
189 lines for Archie Mitchell
293 lines for Max Branning
391 lines for Jack Branning
637 lines for Tanya Branning


In [207]:
final_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,12126.000000,1.212600e+04,1.212600e+04,1.212600e+04,12126.000000,1.212600e+04,12126.000000,12126.000000,1.212600e+04,12126.000000
mean,0.311848,5.546740e-01,5.905356e-01,3.444887e-01,0.437559,1.473839e+00,0.512673,0.827838,2.927435e-01,0.573448
std,0.958388,1.599285e+00,1.788715e+00,1.081884e+00,0.984319,5.798210e+00,1.562307,2.174958,8.507984e-01,1.943453
min,-0.562043,-3.973421e-01,-4.664832e-01,-5.643575e-01,-0.298660,-2.952595e-01,-0.388738,-0.432771,-5.274702e-01,-0.395905
25%,-0.364714,-3.026762e-01,-2.996513e-01,-3.533566e-01,-0.250869,-2.212133e-01,-0.301930,-0.262297,-3.531565e-01,-0.305054
50%,0.000000,2.339166e-17,-3.407105e-17,3.875684e-17,0.000000,3.162398e-17,0.000000,0.000000,-5.006642e-17,0.000000
75%,0.635286,6.973238e-01,7.003487e-01,6.466434e-01,0.749131,7.787867e-01,0.698070,0.737703,6.468435e-01,0.694946
max,4.955059,1.304373e+01,1.907532e+01,1.043664e+01,4.631017,7.215544e+01,25.217867,12.029810,3.065483e+00,38.136620


In [208]:
hf_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,-0.107885,-0.047529,-0.020012,-0.029120,-0.161780,-0.014409,-0.008053,-0.062501,-0.182367,-0.008611
std,0.118304,0.079811,0.033840,0.034640,0.216252,0.047177,0.013951,0.107786,0.189139,0.017247
min,-0.681044,-0.670786,-0.369715,-0.352251,-1.083072,-0.589514,-0.228657,-0.617643,-0.798769,-0.341962
25%,-0.147811,-0.054647,-0.022089,-0.038794,-0.230231,-0.008753,-0.009709,-0.058034,-0.261086,-0.009689
50%,-0.069391,-0.019848,-0.008840,-0.018090,-0.065649,-0.002417,-0.003476,-0.021475,-0.117287,-0.003522
75%,-0.024370,-0.004743,-0.003171,-0.006776,-0.010534,-0.000617,-0.000780,-0.008476,-0.038778,-0.000815
max,-0.000012,-0.000019,-0.000015,-0.000020,-0.000034,-0.000014,-0.000004,-0.000028,-0.000026,-0.000009


In [209]:
hf_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,-0.107885,-0.047529,-0.020012,-0.029120,-0.161780,-0.014409,-0.008053,-0.062501,-0.182367,-0.008611
std,0.118304,0.079811,0.033840,0.034640,0.216252,0.047177,0.013951,0.107786,0.189139,0.017247
min,-0.681044,-0.670786,-0.369715,-0.352251,-1.083072,-0.589514,-0.228657,-0.617643,-0.798769,-0.341962
25%,-0.147811,-0.054647,-0.022089,-0.038794,-0.230231,-0.008753,-0.009709,-0.058034,-0.261086,-0.009689
50%,-0.069391,-0.019848,-0.008840,-0.018090,-0.065649,-0.002417,-0.003476,-0.021475,-0.117287,-0.003522
75%,-0.024370,-0.004743,-0.003171,-0.006776,-0.010534,-0.000617,-0.000780,-0.008476,-0.038778,-0.000815
max,-0.000012,-0.000019,-0.000015,-0.000020,-0.000034,-0.000014,-0.000004,-0.000028,-0.000026,-0.000009
